In [1]:
import os 
os.chdir("C:\\Users\\Microsoft\\AppData\\Local\\Programs\\Python\\Python310\\Scripts\\w6_img")

In [2]:
# setup 

import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

# For reproducibility,
np.random.seed(99)

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

# Q1

Apply histogram equalization on a histogram equalized image. In other words, apply histogram equalization twice on an image. Is there any difference between the first and second output image? You are free to use any image.

In [3]:
img = cv.imread('dog1.jfif', 0)
                
#histogram equalization
equ1 = cv.equalizeHist(img)
equ2 = cv.equalizeHist(equ1)

cv.imshow('output', np.hstack((img, equ1, equ2)))
cv.waitKey(0)
cv.destroyAllWindows()

Q1: There is no difference between first and second output.

# Q2

Load the image 'electronic.jfif'. Then,
Experiment with different kernel size. Which kernel size is the most appropriate?
Perform edge detection using Sobel operator (combined both x and y) with and without image smoothing (Gaussian blurring). Display the 2 images.
Try Laplacian of Gaussian

Based on the exercise conducted from (A) - (C), which image processing pathway is optimal? Justify your answer.

In [4]:
img = cv.imread('electronic.jfif', 0) 
img2 = img.copy() 

sobelx = cv.Sobel(img, cv.CV_64F, 1, 0, ksize = 1) #kernel size = 1
sobely = cv.Sobel(img, cv.CV_64F, 0, 1, ksize = 1)
sobelx2 = cv.Sobel(img2, cv.CV_64F, 1, 0, ksize = 3) #kernel size = 3
sobely2 = cv.Sobel(img2, cv.CV_64F, 0, 1, ksize = 3)
sobelx3 = cv.Sobel(img2, cv.CV_64F, 1, 0, ksize = 5) #kernel size = 5
sobely3 = cv.Sobel(img2, cv.CV_64F, 0, 1, ksize = 5)

# L2-norm formula
grad_mag = cv.magnitude(sobelx, sobely)
grad_mag = cv.convertScaleAbs(grad_mag)
grad_mag2 = cv.magnitude(sobelx2, sobely2)
grad_mag2 = cv.convertScaleAbs(grad_mag2)
grad_mag3 = cv.magnitude(sobelx3, sobely3)
grad_mag3 = cv.convertScaleAbs(grad_mag3)

cv.imshow('result', np.hstack((img, grad_mag, grad_mag2, grad_mag3)))
cv.waitKey(0)
cv.destroyAllWindows()

Kernel size = 1 is the most appropriate.

In [5]:
#Gaussian blurring
img_blur = cv.GaussianBlur(img, (5,5), 0)

sobelx = cv.Sobel(img_blur, cv.CV_64F, 1, 0, ksize = 1)
sobely = cv.Sobel(img_blur, cv.CV_64F, 0, 1, ksize = 1)

img_blur_grad_mag = cv.magnitude(sobelx,sobely)
img_blur_grad_mag = cv.convertScaleAbs(img_blur_grad_mag)

cv.imshow('result', np.hstack((grad_mag, img_blur_grad_mag)))
cv.waitKey(0)
cv.destroyAllWindows()

In [6]:
#Laplacian of Gaussian
im_blur = cv.GaussianBlur(img, (5,5), 0)
laplacian = cv.Laplacian(im_blur,cv.CV_64F, ksize = 3)
laplacian_8u = cv.convertScaleAbs(laplacian)

cv.imshow('result', np.hstack((grad_mag, img_blur_grad_mag, laplacian_8u)))
cv.waitKey(0)
cv.destroyAllWindows()

Q2: Using Sobel operator with Gaussian blurring is optimal for the image processing pathway as the noise from the image shown is reduced and the edge of image is obvious. 

# Q3

Experiment with different edge detectors: Sobel, Laplacian, Prewitt, Scharr derivatives and Canny operators (all with aperture size of 3) on image named 'pineapple.jfif'. Comment on the results.

In [7]:
img = cv.imread('pineapple.jfif', 0)
img_blur = cv.GaussianBlur(img, (5,5), 0) #gaussian blurring 

#Sobel
sobelx = cv.Sobel(img_blur, cv.CV_64F, 1, 0, ksize = 3) #kernel size = 3
sobely = cv.Sobel(img_blur, cv.CV_64F, 0, 1, ksize = 3)
grad_mag = cv.magnitude(sobelx,sobely)
grad_mag = cv.convertScaleAbs(grad_mag)


#Laplacian
laplacian = cv.Laplacian(img_blur, cv.CV_64F, ksize = 3)
laplacian = cv.convertScaleAbs(laplacian)


#Prewitt operator
#Horizontal edges or along the x-axis, Vertical Edges or along the y-axis
kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]]) 
img_prewittx = cv.filter2D(img_blur, -1, kernelx)
img_prewitty = cv.filter2D(img_blur, -1, kernely)
img_prewitt = (img_prewittx + img_prewitty) #combine


#Scharr operator
#identify and highlight gradient edges/features using the first derivative
scharr_X = cv.Scharr(img_blur, cv.CV_64F, 1, 0) 
scharr_X_abs = np.uint8(np.absolute(scharr_X)) 
scharr_Y = cv.Scharr(img_blur, cv.CV_64F, 0, 1) 
scharr_Y_abs = np.uint8(np.absolute(scharr_Y)) 
img_scharr = cv.bitwise_or(scharr_Y_abs,scharr_X_abs) 


#Canny operators
img_canny = cv.Canny(img_blur, 100, 200)


cv.imshow('Sobel/Laplacian/Prewitt/Scharr/Canny', np.hstack((grad_mag, laplacian, img_prewitt, img_scharr, img_canny)))
cv.waitKey(0)
cv.destroyAllWindows()

Q3: The best and clearest edge detected is by using canny operator, followed by sobel operator. Prewitt operator detect blurry edges, Laplacian operator detect edge but not very obvious and Scharr edge detection is quite messy and includes noise. 

# Q4

Write a program to identify the white object (probably laptop) present in the image 'electronic.jfif'. Draw bounding boxes on the objects.

In [8]:
img = cv.imread('electronic.jfif', 0)

img = cv.GaussianBlur(img, (5,5), 0)
ret, th = cv.threshold(img, 180, 255, cv.THRESH_BINARY)

contours, hierarchy = cv.findContours(th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

for i in contours:
    area = cv.contourArea(i)
    if area >= 1000:
        cnt = i
        x, y, w, h = cv.boundingRect(cnt)
        cv.rectangle(img, (x,y), (x+w, y+h), (0, 255, 255), 2)
    else:
        continue

img_bgr = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
cv.imshow('result', img_bgr)
cv.waitKey(0)
cv.destroyAllWindows()